In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle

from models import Models
from morph import Morph
from generation import Generation

In [ ]:
# shift values = [0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.0]
shift_values = np.arange(0.5, -0.01, -0.05).tolist()

results = {}
for s in shift_values:
    for b in [0.5, 0.10, 0.15, 0.20]:
        # Generate dataset with shift
        gen = Generation(base_functions=('sin', 'sin'), n_samples=100, frequencies=(0.5,0.5), noise_level=(0.05, 0.05), shift_vert=s, shif_horz=s) 
        X, y = gen.generate_data()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train models
        lstm = Models('lstm', X_train, y_train)
        lstm.train_lstm()
        catch22 = Models('catch22', X_train, y_train)
        catch22.train_catch22()
        rocket = Models('rocket', X_train, y_train)
        rocket.train_rocket()

        # Calculate morphing
        res = {}
        morph = Morph(X_test, y_test)
        morph.get_DTWGlobalBorderline(perc_samples=b) 

        res['lsmt'] = morph.Binay_MorphingCalculater(lstm)
        res['catch22'] = morph.Binay_MorphingCalculater(catch22)
        res['rocket'] = morph.Binay_MorphingCalculater(rocket)

        results[s] = res

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
with open('results/bord_perc_shift_perf.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
with open('results/bord_perc_shift_perf.pkl', 'rb') as f:
    loaded_res = pickle.load(f)

In [ ]:
# Create lists to store the data
data = []
for shift_val in loaded_res.keys():
    row_data = {'shift_value': float(f"{shift_val:.2f}")}
    for model in loaded_res[shift_val].keys():
        row_data[f'{model}_mean'] = loaded_res[shift_val][model][3]['mean']
        row_data[f'{model}_std'] = loaded_res[shift_val][model][3]['std']
    data.append(row_data)

df = pd.DataFrame(data)
df = df.set_index('shift_value')
df.columns = pd.MultiIndex.from_tuples([
    (col.split('_')[0], col.split('_')[1]) 
    for col in df.columns
])

df.to_csv("results/results_sin_cos.csv")
df